In [1]:
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torch.cuda.amp import GradScaler, autocast

import torch.nn.functional as F
from torch.nn import Conv2d, BatchNorm2d, ReLU, Sigmoid


if torch.cuda.is_available():
    print("CUDA is available. PyTorch is running on the GPU.")
    print("Device name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available. PyTorch is running on the CPU.")

CUDA is available. PyTorch is running on the GPU.
Device name: Quadro RTX 8000


# Import Data

In [3]:
data_dir = '/vast/xj2173/diffeo/data/temp_inv_data/'
numbers = [f"{i:02}" for i in range(40)]
data_name = [data_dir + f'15-100-4-4-3-224-224_image-00{i}_activation_layer-09.pt' for i in numbers]

In [4]:
# data[0] is the 0th picture, data[1] is the 1st picture, etc..
data = [torch.load(file_name, map_location='cpu') for file_name in tqdm(data_name)]
data = torch.stack(data, dim=0)
data.shape

100%|██████████| 40/40 [01:09<00:00,  1.75s/it]


torch.Size([40, 1500, 48, 56, 56])

- 40 different images
- 1500 different diffeos
    - 15 diffeo strengths
    - 100 individual diffeos per diffeo strength
- 48 channels
- 56 x 56 image

In [14]:
target_channel = 1
num_of_features_cutoff = 1500

features = data[:, 1:, target_channel, :, :]
pics, diffeo, size, _ = features.shape
features = features.permute(1, 0, 2, 3)
features = features.reshape(pics * diffeo, size, size)
features = features[:num_of_features_cutoff, :, :]

class NaiveInverseDiffeo(Dataset):
    def __init__(self, transform=None):
        self.label = data[target_picture, 0, target_channel, :, :]
        # self.num_pics = pics
        
    def __len__(self):
        return int(features.shape[0])

    def __getitem__(self, idx):
        # num_pics = self.num_pics
        label = self.label
        feature = features[idx]
        
        return feature, label


dataset = NaiveInverseDiffeo()

In [15]:
features[0].shape

torch.Size([56, 56])

# Objective

We've experimentally confirmed:
$$
g_{naive}^{-1} \star N_i (g \cdot I) = N_i(I) + \eta_i
$$
Where $g_{naive}^{-1}$ is just the inverse of $g$ using the $\cdot$ algebra. So basically, it looks like the original but w/ some noise $\eta_i$. 

Now the objective is to see if we can learn $\eta_i$. Or rewritten in another way
$$
h^{-1} \star g_{naive}^{-1} \star  N_i (g \cdot I) = N_i(I)
$$
We want to learn $h^{-1}$, s.t. the 2nd equation is true


# Learning $h^{-1}$

- 40 different images
- 15 diffeo strengths
- 100 individual diffeos per diffeo strength
- 48 channels
- 56 x 56 image

### Split Data

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [17]:
split = 0.8
batch_size = 25

train_size = int(split * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(dataset=val_dataset,   batch_size=batch_size, shuffle=True)

### Initalize Model

In [65]:
class Model0(nn.Module):
    def __init__(self, batch_size):
        super(Model0, self).__init__()
        self.batch_size = batch_size  # Store batch_size if needed for some reason
        
        self.conv1 = nn.Conv2d(self.batch_size, 16, kernel_size=3, stride=1, padding=1)  
        self.conv2 = nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(1568, 100)  # Adjust the input size according to the output size after pooling
        self.fc2 = nn.Linear(100, 3136)

    def forward(self, x):
        # Convoultion Layer
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)  # Apply 2x2 max pooling
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)  # Apply 2x2 max pooling

        # MLP
        x = torch.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = x.reshape(1, 56, 56)
        return x


In [ ]:
learning_rate = 0.001
num_epochs = 10000

model = Model0(batch_size)
model = model.to(device)

criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum = 0.4)
scaler = GradScaler()

print("Starting training...")
for epoch in tqdm(range(num_epochs)):
    model.train()  # Set the model to training mode
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        # Move inputs and targets to the GPU
        inputs, targets = inputs.to(device), targets.to(device)
    
        with autocast():
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
        
        optimizer.zero_grad()  # Zero the gradient buffers
        scaler.scale(loss).backward()  # Backpropagation
        scaler.step(optimizer)  # Update weights
        scaler.update()

    if (epoch + 1) % 25 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


Starting training...


  0%|          | 26/10000 [00:04<28:05,  5.92it/s]

Epoch [25/10000], Loss: 27.0307


  1%|          | 51/10000 [00:08<29:49,  5.56it/s]

Epoch [50/10000], Loss: 25.3266


...

 33%|███▎      | 3251/10000 [09:32<19:27,  5.78it/s]

Epoch [3250/10000], Loss: 23.8516


 33%|███▎      | 3276/10000 [09:36<20:02,  5.59it/s]

Epoch [3275/10000], Loss: 24.2992


 33%|███▎      | 3292/10000 [09:39<19:57,  5.60it/s]

Things I've tried...
- Changed the minimization method
  - Adam
  - SGD (w/ various momentum: `0.1`, `0.4`)

In [66]:
class Model1(nn.Module):
    def __init__(self):
        super(Model1, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(48, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 48, kernel_size=3, padding=1),
            nn.BatchNorm2d(48),
            nn.ReLU())
        
        self.final_conv = nn.Conv2d(48, 48, kernel_size=1)  # Maintain the channel dimension
        
        self.fc1 = nn.Linear(48 * 56 * 56, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 48 * 56 * 56)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.final_conv(out)
        
        out = out.view(out.size(0), -1)  # Flatten the tensor
        out = nn.ReLU()(self.fc1(out))
        out = nn.ReLU()(self.fc2(out))
        out = self.fc3(out)
        
        out = out.view(out.size(0), 48, 56, 56)  # Reshape to the original shape
        return out